## **Cleaning not-null values**

After dealing with many datasets I can tell you that "missing data" is not such a big deal. The best thing that can happen is to clearly see values like np.nan. The only thing you need to do is just use methods like isnull and fillna/dropna and pandas will take care of the rest.

But sometimes, you can have invalid values that are not just "missing data" (None, or nan). For example:

---

### Hands on!!

In [2]:
# importing stuffs
import numpy as np
import pandas as pd

#### What we will do here ?
- here we will clean the data based on certain parametres:
   - previously removing null values
   - here finding duplicates
   - here removing duplicates
   - here checking irrelevant values like age > 100 or some other not relevent values
   - here performing datatype correction
   - here modifying incorrect notation and correcting it

In [3]:
# getting the data as dataframe
df = pd.DataFrame({
    'Sex': ['M', 'F', 'F', 'D', '?'],
    'Age': [29, 30, 24, 290, 25],
})
df

,Sex,Age
0,M,29
1,F,30
2,F,24
3,D,290
4,?,25


The previous DataFrame doesn't have any "missing value", but clearly has invalid data. 290 doesn't seem like a valid age, and D and ? don't correspond with any known sex category. How can you clean these not-missing, but clearly invalid values then?

### Finding Unique Values

The first step to clean invalid values is to notice them, then identify them and finally handle them appropriately (remove them, replace them, etc). Usually, for a "categorical" type of field (like Sex, which only takes values of a discrete set ('M', 'F')), we start by analyzing the variety of values present. For that, we use the unique() method:

#### df["value"].unique() 
- returns a NumPy array of all unique values in the specified column "value".
- Removes duplicates and preserves the original order in which they appear.

In [4]:
df["Sex"].unique()

array(['M', 'F', 'D', '?'], dtype=object)

#### df['Sex'].value_counts() 
- counts the occurrences of each unique value in the "Sex" column.

#### Returns a Series where:
- The index contains unique values from the column.
- The values represent their frequency (count) in the DataFrame.

In [5]:
df["Sex"].value_counts()

Sex
F    2
M    1
D    1
?    1
Name: count, dtype: int64

Clearly if you see values like 'D' or '?', it'll immediately raise your attention. Now, what to do with them? Let's say you picked up the phone, called the survey company and they told you that 'D' was a typo and it should actually be F. You can use the replace function to replace these values:

#### 🔹 .replace() Syntax and Usage in Pandas
- The .replace() method in Pandas is used to replace specific values in a DataFrame or Series. It works on both exact matches and pattern-based replacements using regex.
#### ✅ Basic Syntax
- **df.replace(to_replace, value, inplace=False, regex=False)**
  
#### 🔎 Parameters:  
| Parameter   | Description |
|------------|-------------|
| `to_replace` | The value (or list/dictionary) to be replaced |
| `value`      | The new value to replace with |
| `inplace`    | If `True`, modifies the DataFrame in place (default: `False`) |
| `regex`      | If `True`, treats `to_replace` as a regex pattern |

In [6]:
# here we will be replacing the sex value D with F
df["Sex"].replace('D','F')

0    M
1    F
2    F
3    F
4    ?
Name: Sex, dtype: object

It can accept a dictionary of values to replace. For example, they also told you that there might be a few 'N's, that should actually be 'M's:

In [7]:
df["Sex"].replace({'D' : 'F', '?' : 'M'})

0    M
1    F
2    F
3    F
4    M
Name: Sex, dtype: object

In [8]:
df

,Sex,Age
0,M,29
1,F,30
2,F,24
3,D,290
4,?,25


Note that : all these changes are not to the original hence the original df is not modified

If you have many columns to replace, you could apply it at "DataFrame level":

In [9]:
df.replace({
    'Sex' : {
        'D' : 'F',
        '?' : 'M'
    },
    'Age' : {
        290 : 29
    }
})

,Sex,Age
0,M,29
1,F,30
2,F,24
3,F,29
4,M,25


In the previous example, I explicitly replaced 290 with 29 (assuming it was just an extra 0 entered at data-entry phase). But what if you'd like to remove all the extra 0s from the ages columns? (example, 150 > 15, 490 > 49).

The first step would be to just set the limit of the "not possible" age. Is it 100? 120? Let's say that anything above 100 isn't credible for our dataset. We can then combine boolean selection with the operation:

In [10]:
df[df["Age"] > 100]

,Sex,Age
3,D,290


And we can now just divide by 10:

In [11]:
df["Age"] > 100

0    False
1    False
2    False
3     True
4    False
Name: Age, dtype: bool

In [12]:
df.loc[df["Age"] > 100, "Age"] = df.loc[df["Age"] > 100, "Age"]/10

In [13]:
df

,Sex,Age
0,M,29
1,F,30
2,F,24
3,D,29
4,?,25


#### **🔹 Step-by-Step Breakdown:**
---

**1️⃣ df['Age'] > 100**
- This creates a Boolean mask that identifies rows where "Age" is greater than 100.

**2️⃣ df.loc[df['Age'] > 100, 'Age']**
- This selects the "Age" column only for those rows where "Age" is greater than 100.

**3️⃣ df.loc[df['Age'] > 100, 'Age'] / 10**
- This divides those values by 10 to correct wrongly recorded ages.

**4️⃣ Assignment =**
- The calculated values replace the original values in the "Age" column.

### Duplicates

Checking duplicate values is extremely simple. It'll behave differently between Series and DataFrames. Let's start with Series. As an example, let's say we're throwing a fancy party and we're inviting Ambassadors from Europe. But can only invite one ambassador per country. This is our original list, and as you can see, both the UK and Germany have duplicated ambassadors:

In [14]:
ambassadors = pd.Series([
    'France',
    'United Kingdom',
    'United Kingdom',
    'Italy',
    'Germany',
    'Germany',
    'Germany',
], index=[
    'Gérard Araud',
    'Kim Darroch',
    'Peter Westmacott',
    'Armando Varricchio',
    'Peter Wittig',
    'Peter Ammon',
    'Klaus Scharioth '
])
ambassadors

Gérard Araud                  France
Kim Darroch           United Kingdom
Peter Westmacott      United Kingdom
Armando Varricchio             Italy
Peter Wittig                 Germany
Peter Ammon                  Germany
Klaus Scharioth              Germany
dtype: object

here we are tasked to filter the ambassador series such that each country is represented by a single ambassador

In [15]:
ambassadors.unique()

array(['France', 'United Kingdom', 'Italy', 'Germany'], dtype=object)

In [16]:
ambassadors.value_counts()

Germany           3
United Kingdom    2
France            1
Italy             1
Name: count, dtype: int64

The two most important methods to deal with duplicates are duplicated (that will tell you which values are duplicates) and drop_duplicates (which will just get rid of duplicates):

**🔹 duplicated() Function in Pandas**  
The .duplicated() function is used to identify duplicate rows in a Pandas DataFrame or Series.

**✅ Syntax**

- df.duplicated(subset=None, keep='first')

| Parameter  | Description |
|------------|-------------|
| `subset`   | Column(s) to check for duplicates (default: all columns) |
| `keep`     | `'first'` (keep first occurrence), `'last'` (keep last), `False` (mark all as duplicates) |

In [17]:
ambassadors.duplicated()

Gérard Araud          False
Kim Darroch           False
Peter Westmacott       True
Armando Varricchio    False
Peter Wittig          False
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

In this case duplicated didn't consider 'Kim Darroch', the first instance of the United Kingdom or 'Peter Wittig' as duplicates. That's because, by default, it'll consider the first occurrence of the value as not-duplicate. You can change this behavior with the keep parameter:

In [18]:
# suppose we want to keep the last then we will use the keep = "last" attribute
ambassadors.duplicated(keep= "last")

Gérard Araud          False
Kim Darroch            True
Peter Westmacott      False
Armando Varricchio    False
Peter Wittig           True
Peter Ammon            True
Klaus Scharioth       False
dtype: bool

In this case, the result is "flipped", 'Kim Darroch' and 'Peter Wittig' (the first ambassadors of their countries) are considered duplicates, but 'Peter Westmacott' and 'Klaus Scharioth' are not duplicates. You can also choose to mark all of them as duplicates with keep=False:

In [19]:
ambassadors.duplicated(keep= False)

Gérard Araud          False
Kim Darroch            True
Peter Westmacott       True
Armando Varricchio    False
Peter Wittig           True
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

A similar method is drop_duplicates, which just excludes the duplicated values and also accepts the keep parameter:

The **.drop_duplicates()** function is used to remove duplicate rows from a DataFrame. It keeps only one occurrence of the duplicate rows based on the keep parameter.

In [22]:
# note that default keep = "first"
ambassadors.drop_duplicates(keep = "first")

Gérard Araud                  France
Kim Darroch           United Kingdom
Armando Varricchio             Italy
Peter Wittig                 Germany
dtype: object

In [23]:
ambassadors.drop_duplicates(keep = "last")

Gérard Araud                  France
Peter Westmacott      United Kingdom
Armando Varricchio             Italy
Klaus Scharioth              Germany
dtype: object

In [25]:
ambassadors.drop_duplicates(keep = False)
# it removes all occurrences of duplicate rows, leaving only the unique rows.

Gérard Araud          France
Armando Varricchio     Italy
dtype: object

### Duplicates in DataFrames

Conceptually speaking, duplicates in a DataFrame happen at "row" level. Two rows with exactly the same values are considered to be duplicates:

In [26]:
players = pd.DataFrame({
    'Name': [
        'Kobe Bryant',
        'LeBron James',
        'Kobe Bryant',
        'Carmelo Anthony',
        'Kobe Bryant',
    ],
    'Pos': [
        'SG',
        'SF',
        'SG',
        'SF',
        'SF'
    ]
})

In [27]:
players

,Name,Pos
0,Kobe Bryant,SG
1,LeBron James,SF
2,Kobe Bryant,SG
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


In the previous DataFrame, we clearly see that Kobe is duplicated; but he appears with two different positions. What does duplicated say?

In [28]:
players.duplicated()

0    False
1    False
2     True
3    False
4    False
dtype: bool

Again, conceptually, "duplicated" means "all the column values should be duplicates". We can customize this with the subset parameter:

The subset parameter in .duplicated() allows you to check for duplicates based only on specific columns instead of the entire DataFrame.

In [30]:
# here the duplicated function is performed only on the name column and not the entire dataframe
players.duplicated(subset=['Name'])

0    False
1    False
2     True
3    False
4     True
dtype: bool

And the same rules of keep still apply:

In [31]:
players.duplicated(subset=['Name'], keep='last')

0     True
1    False
2     True
3    False
4    False
dtype: bool

drop_duplicates takes the same parameters:

In [32]:
players.drop_duplicates()

,Name,Pos
0,Kobe Bryant,SG
1,LeBron James,SF
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


In [33]:
players.drop_duplicates(subset = "Name")

,Name,Pos
0,Kobe Bryant,SG
1,LeBron James,SF
3,Carmelo Anthony,SF


In [34]:
players.drop_duplicates(subset=['Name'], keep='last')

,Name,Pos
1,LeBron James,SF
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


### Text Handling

Cleaning text values can be incredibly hard. Invalid text values involves, 99% of the time, mistyping, which is completely unpredictable and doesn't follow any pattern. Thankfully, it's not so common these days, where data-entry tasks have been replaced by machines. Still, let's explore the most common cases:

#### Splitting Columns

The result of a survey is loaded and this is what you get:

In [36]:
df = pd.DataFrame({
    'Data': [
        '1987_M_US _1',
        '1990?_M_UK_1',
        '1992_F_US_2',
        '1970?_M_   IT_1',
        '1985_F_I  T_2'
]})

In [37]:
df

,Data
0,1987_M_US _1
1,1990?_M_UK_1
2,1992_F_US_2
3,1970?_M_ IT_1
4,1985_F_I T_2


You know that the single columns represent the values "year, Sex, Country and number of children", but it's all been grouped in the same column and separated by an underscore. Pandas has a convenient method named split that we can use in these situations:

**🔍 Understanding .str in Pandas**  
In Pandas, .str is a string accessor that allows you to apply string operations on a Pandas Series containing text data.
It enables you to use Python's built-in string methods vectorized across all rows of a column.

**Why Use .str in Pandas?**  
Normally, Python string methods like .split(), .lower(), or .replace() work only on individual strings.
But in Pandas, we deal with entire columns, so .str allows us to apply these methods efficiently.

#### ✅ Summary Table of `.str` Functions

| **Function**               | **Description**                              |
|----------------------------|----------------------------------------------|
| `.str.lower()`             | Converts to lowercase                        |
| `.str.upper()`             | Converts to uppercase                        |
| `.str.strip()`             | Removes leading/trailing spaces              |
| `.str.replace(old, new)`   | Replaces text                                |
| `.str.contains(substring)` | Checks if substring exists                   |
| `.str.split(delimiter)`    | Splits string into a list                    |
| `.str.extract(pattern)`    | Extracts part of a string using regex        |
| `.str.count(substring)`    | Counts occurrences of a substring            |

Note that .split() and other method works with only string hence we need to change the data into a string form 

In [42]:
df["Data"].str.split("_")

0       [1987, M, US , 1]
1       [1990?, M, UK, 1]
2        [1992, F, US, 2]
3    [1970?, M,    IT, 1]
4      [1985, F, I  T, 2]
Name: Data, dtype: object

In [44]:
df['Data'].str.split('_', expand=True)
# expand=True	Expands the split parts into separate columns (returns a DataFrame instead of a list).

,0,1,2,3
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [45]:
df

,Data
0,1987_M_US _1
1,1990?_M_UK_1
2,1992_F_US_2
3,1970?_M_ IT_1
4,1985_F_I T_2


Note that : these changes are not in the original dataframe hence we need to update the changes in the original

In [46]:
df = df['Data'].str.split('_', expand=True)

In [47]:
df

,0,1,2,3
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [48]:
# here we are naming the columns
df.columns = ['Year', 'Sex', 'Country', 'No Children']

In [49]:
df

,Year,Sex,Country,No Children
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [50]:
df['Year'].str.contains('\?')

0    False
1     True
2    False
3     True
4    False
Name: Year, dtype: bool

**✅ Syntax & Explanation**
- .str.contains(pattern)	Checks if each string in the column contains the given pattern.
- '\?'	The escaped question mark (?), meaning we are searching for ? in the text.

In the above line we can see there are some true values popping up  
Meaning we need to remove those  ? from the back of the year

contains takes a regex/pattern as first value, so we need to escape the ? symbol as it has a special meaning for these patterns. Regular letters don't need escaping:

In [52]:
df['Country'].str.contains('U')

0     True
1     True
2     True
3    False
4    False
Name: Country, dtype: bool

Removing blank spaces (like in 'US ' or 'I  T' can be achieved with strip (lstrip and rstrip also exist) or just replace:

Note that we also need to remove unnecessary spaces while cleaning

In [57]:
# removing blank spaces using .strip()
df["Country"].str.strip()

0      US
1      UK
2      US
3      IT
4    I  T
Name: Country, dtype: object

#### 📌 What is Regex (Regular Expressions)?

**Regular Expressions (regex)** are patterns used to **match strings**. They help in **searching, replacing, and extracting** specific text patterns from data.

🔹 **Think of regex as a flexible search tool!**

---

##### ✅ Why Use Regex?
- **Find specific patterns** (e.g., email addresses, phone numbers, dates).  
- **Replace text dynamically** (e.g., remove special characters).  
- **Validate inputs** (e.g., check if a string is a valid email).  
- **Extract relevant data** (e.g., extract digits from a string).  

---

#### 📌 Comprehensive Regex Pattern Guide  

##### **🔹 Basic Character Matching**
| **Pattern**  | **Matches** |
|-------------|------------|
| `.`         | Any **single** character (except newline). |
| `\d`        | Any **digit** (`0-9`). |
| `\D`        | Any **non-digit** character. |
| `\w`        | Any **word character** (`A-Z, a-z, 0-9, _`). |
| `\W`        | Any **non-word** character. |
| `\s`        | Any **whitespace** (space, tab, newline). |
| `\S`        | Any **non-whitespace** character. |

---

##### **🔹 Anchors**
| **Pattern**  | **Matches** |
|-------------|------------|
| `^`         | **Start** of a string. |
| `$`         | **End** of a string. |
| `\b`        | **Word boundary** (e.g., `\bword\b` matches "word" but not "wording"). |
| `\B`        | **Not** a word boundary. |

---

##### **🔹 Quantifiers**
| **Pattern**  | **Matches** |
|-------------|------------|
| `*`         | **0 or more occurrences**. |
| `+`         | **1 or more occurrences**. |
| `?`         | **0 or 1 occurrence**. |
| `{n}`       | **Exactly `n` occurrences**. |
| `{n,}`      | **At least `n` occurrences**. |
| `{n,m}`     | **Between `n` and `m` occurrences**. |

---

##### **🔹 Grouping & Capturing**
| **Pattern**  | **Matches** |
|-------------|------------|
| `(abc)`     | **Capturing group**: Matches `"abc"` and saves it for reference. |
| `(?:abc)`   | **Non-capturing group**: Matches `"abc"` but **does not save** it. |
| `(?P<name>abc)` | **Named capturing group**: Matches `"abc"` and saves it as `"name"`. |

---

##### **🔹 Alternation & Character Classes**
| **Pattern**  | **Matches** |
|-------------|------------|
| `a|b`       | Either `"a"` **or** `"b"`. |
| `[abc]`     | Any **one** of `"a"`, `"b"`, or `"c"`. |
| `[^abc]`    | **Any character except** `"a"`, `"b"`, or `"c"`. |
| `[a-z]`     | Any **lowercase letter**. |
| `[A-Z]`     | Any **uppercase letter**. |
| `[0-9]`     | Any **digit** (`0-9`). |

---

##### **🔹 Lookaheads & Lookbehinds**
| **Pattern**  | **Matches** |
|-------------|------------|
| `(?=abc)`   | **Positive lookahead**: Ensures `"abc"` is ahead, but doesn't consume it. |
| `(?!abc)`   | **Negative lookahead**: Ensures `"abc"` is **not** ahead. |
| `(?<=abc)`  | **Positive lookbehind**: Ensures `"abc"` is before, but doesn't consume it. |
| `(?<!abc)`  | **Negative lookbehind**: Ensures `"abc"` is **not** before. |

---

##### **🔹 Special Cases**
| **Pattern**  | **Matches** |
|-------------|------------|
| `\t`        | Tab character. |
| `\n`        | Newline character. |
| `\\`        | Matches a **literal** backslash (`\`). |
| `\(` `\)`   | Matches **literal** parentheses. |

---

#### **✅ Example Usage**
```python
import re  

text = "My email is example123@mail.com and my phone is 987-654-3210."  
email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"  
phone_pattern = r"\d{3}-\d{3}-\d{4}"  

email_match = re.search(email_pattern, text)  
phone_match = re.search(phone_pattern, text)  

print(email_match.group())  # Output: example123@mail.com  
print(phone_match.group())  # Output: 987-654-3210  



In [54]:
df['Country'].str.replace(' ', '')

0    US
1    UK
2    US
3    IT
4    IT
Name: Country, dtype: object

As we said, replace and contains take regex patterns, which can make it easier to replace values in bulk:

In [60]:
df['Year'] = df['Year'].str.replace(r'(?P<year>\d{4})\?', lambda m: m.group('year'), regex=True)

In [61]:
df

,Year,Sex,Country,No Children
0,1987,M,US,1
1,1990,M,UK,1
2,1992,F,US,2
3,1970,M,IT,1
4,1985,F,I T,2


##### 📌 Explanation
- regex=True: Ensures that the first argument is treated as a regular expression.  
- r'(?P<year>\d{4})\?': Finds 4-digit years followed by ?.  
- lambda m: m.group('year'): Replaces each match with only the captured year (removing ?).  

But, be warned:   

Some people, when confronted with a problem, think "I know, I'll use regular expressions." Now they have two problems.  

As you can see, all these string/text-related operations are applied over the str attribute of the series. That's because they have a special place in Series handling and you can read more about it here.